In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!pip install bs4

from bs4 import BeautifulSoup

You should consider upgrading via the 'c:\users\axew2\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=a3be08516ae2073104588b6b41ff604c667241945e7160c1042c7b43cad74dd9
  Stored in directory: c:\users\axew2\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


You should consider upgrading via the 'c:\users\axew2\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Take the URL from wiki to start Wrapping the data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
data = requests.get(url).text

Use the Beautiful Soup Library to take the Table about Postal Code of Toronto from Wikipeadia

In [8]:
soup = BeautifulSoup(data, "html.parser")

In [9]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Show first 5 heading of the Postal Code

In [63]:
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [11]:
df.shape

(103, 3)

We can see that there are 103 Postal Code with Assigned 

In [12]:
!pip install geocoder

You should consider upgrading via the 'c:\users\axew2\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=d47ecb213aa13852c217c4e45c9a2cc98577227ac934a042de8c8445c6bcfc28
  Stored in directory: c:\users\axew2\appdata\local\pip\cache\wheels\56\b0\fe\4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future


In [13]:
import geocoder

Because When using the library Geocoder to take Latitude and Longitude it always return None so I use the Geospatial_Coordinates to add to my Dataframe. It has Latitude and Longitude of the Postal Code in Toronto

In [17]:
data = pd.read_csv('Geospatial_Coordinates (1).csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
data.shape

(103, 3)

Rename the column Postal Code by PostalCode to has the same name with the Dataframe and merge together

In [50]:
data = data.rename(columns = {'Postal Code': 'PostalCode'})

In [51]:
data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
new_df = pd.merge(df,data, on= "PostalCode")

In [62]:
new_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [92]:
data_toronto = new_df[new_df['Borough'].isin([i for i in new_df['Borough'] if 'Toronto' in i])]

Take the dataframe with only Borough contain word Toronto

In [227]:
data_toronto.iloc[38,0]

'M7Y'

In [228]:
data_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Berczy Park,43.654260,-79.360636
9,M5B,Downtown Toronto,"Brockton, Parkdale Village, Exhibition Place",43.657162,-79.378937
15,M5C,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.651494,-79.375418
19,M4E,East Toronto,Central Bay Street,43.676357,-79.293031
20,M5E,Downtown Toronto,Christie,43.644771,-79.373306
24,M5G,Downtown Toronto,Church and Wellesley,43.657952,-79.387383
25,M6G,Downtown Toronto,"Commerce Court, Victoria Hotel",43.669542,-79.422564
30,M5H,Downtown Toronto,Davisville,43.650571,-79.384568
31,M6H,West Toronto,Davisville North,43.669005,-79.442259
35,M4J,East York/East Toronto,"Dufferin, Dovercourt Village",43.685347,-79.338106


In [200]:
print(a)

None


In [94]:
address = 'Toronto City, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [101]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data_toronto['Borough'].unique()),
        data_toronto.shape[0]
    )
)

The dataframe has 7 boroughs and 39 neighborhoods.


Take the Latitude and Longitude of Toronto and generate maps to visualize the Borough in Toronto

In [105]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_toronto['Latitude'], data_toronto['Longitude'], data_toronto['Borough'], data_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [102]:
CLIENT_ID = 'HSXLXVHWZWBXY0R1DTY50VWZDCDHQ3YCIJRZ5ZTCM5XXEVWR' #  Foursquare ID
CLIENT_SECRET = 'BLVQCC40KCDBSXBE5TFC1Y2R1OXFW41FCRBK4UHKWFEJ4ZBG' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HSXLXVHWZWBXY0R1DTY50VWZDCDHQ3YCIJRZ5ZTCM5XXEVWR
CLIENT_SECRET:BLVQCC40KCDBSXBE5TFC1Y2R1OXFW41FCRBK4UHKWFEJ4ZBG


In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name,)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [229]:
borough_venues = getNearbyVenues(names=data_toronto['Neighborhood'],
                                   latitudes=data_toronto['Latitude'],
                                   longitudes=data_toronto['Longitude']
                                  )

Berczy Park
Brockton, Parkdale Village, Exhibition Place
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Central Bay Street
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Davisville
Davisville North
Dufferin, Dovercourt Village
Enclave of M4L
Enclave of M5E
First Canadian Place, Underground city
Forest Hill North & West
Garden District, Ryerson
Harbourfront East, Union Station, Toronto Islands
High Park, The Junction South
India Bazaar, The Beaches West
Kensington Market, Chinatown, Grange Park
Lawrence Park
Little Portugal, Trinity
Moore Park, Summerhill East
North Toronto West
Parkdale, Roncesvalles
Regent Park, Harbourfront
Richmond, Adelaide, King
Rosedale
Roselawn
Runnymede, Swansea
St. James Town
St. James Town, Cabbagetown
Studio District
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
The Annex, North Midtown, Yorkville
The Beaches
The Danforth  East
The Danforth West, Riverdale
Toronto D

In [230]:
borough_venues.shape

(1571, 7)

In [232]:
borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,45,45,45,45,45,45
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",82,82,82,82,82,82
Central Bay Street,4,4,4,4,4,4
Christie,59,59,59,59,59,59
Church and Wellesley,64,64,64,64,64,64
"Commerce Court, Victoria Hotel",16,16,16,16,16,16
Davisville,92,92,92,92,92,92
Davisville North,16,16,16,16,16,16


In [248]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 232 uniques categories.


Using one hot encoding to convert all Categories to 0 and 1

In [249]:
# one hot encoding
data_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
data_onehot['Neighbor'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [data_onehot.columns[-1]] + list(data_onehot.columns[:-1])
data_onehot = data_onehot[fixed_columns]

data_onehot.head()

,Neighbor,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [250]:
data_onehot.shape

(1571, 233)

In [251]:
data_grouped = data_onehot.groupby('Neighbor').mean().reset_index()
data_grouped.shape

(38, 233)

In [252]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [295]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbor']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbor'] = data_grouped['Neighbor']

for ind in np.arange(data_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(data_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbor,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Shoe Store
1,"Brockton, Parkdale Village, Exhibition Place",Clothing Store,Coffee Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Fast Food Restaurant,Movie Theater,Japanese Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Beer Bar,Gym,Hotel,Farmers Market,Italian Restaurant,Lingerie Store
3,Central Bay Street,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
4,Christie,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Farmers Market,Seafood Restaurant,Juice Bar


After some trial, I use kmeans with k = 2

In [294]:
kclusters = 2

data_grouped_clustering = data_grouped.drop('Neighbor', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [255]:
data_toronto = data_toronto[data_toronto['Neighborhood'] != 'St. James Town']

In [296]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [297]:
data_merged = data_toronto
# merge data_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data_merged.join(neighborhoods_venues_sorted.set_index('Neighbor'), on='Neighborhood')

In [298]:
data_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Berczy Park,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Shoe Store
9,M5B,Downtown Toronto,"Brockton, Parkdale Village, Exhibition Place",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Fast Food Restaurant,Movie Theater,Japanese Restaurant
15,M5C,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Beer Bar,Gym,Hotel,Farmers Market,Italian Restaurant,Lingerie Store
19,M4E,East Toronto,Central Bay Street,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
20,M5E,Downtown Toronto,Christie,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Farmers Market,Seafood Restaurant,Juice Bar


In [299]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood'], data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

From the map we can see that these Venue near the large street and in central of the City. People like to go Coffe the most so we Group these Venue into 1 Group.

In [300]:
data_merged.loc[data_merged['Cluster Labels'] == 0, data_merged.columns[[1] + list(range(3, data_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Shoe Store
9,Downtown Toronto,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Fast Food Restaurant,Movie Theater,Japanese Restaurant
15,Downtown Toronto,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Beer Bar,Gym,Hotel,Farmers Market,Italian Restaurant,Lingerie Store
19,East Toronto,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
20,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Farmers Market,Seafood Restaurant,Juice Bar
24,Downtown Toronto,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Department Store,Thai Restaurant
25,Downtown Toronto,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Baby Store,Italian Restaurant,Candy Store,Athletics & Sports,Restaurant,Nightclub
30,Downtown Toronto,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Hotel,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Burrito Place,Pizza Place
31,West Toronto,43.669005,-79.442259,0,Pharmacy,Bakery,Art Gallery,Café,Middle Eastern Restaurant,Bar,Supermarket,Bank,Pool,Coffee Shop
36,Downtown Toronto,43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Sporting Goods Shop,Scenic Lookout,Restaurant,Fried Chicken Joint,Brewery,Italian Restaurant


# Cluster 2

So with Cluster 2, We can see that the 1st Most Common Venue is Park and People in there like to go to Yoga Studio. So we group 4 Neighbor into 1 Group.

In [301]:
data_merged.loc[data_merged['Cluster Labels'] == 1, data_merged.columns[[1] + list(range(3, data_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,43.685347,-79.338106,1,Park,Convenience Store,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
61,Central Toronto,43.728020,-79.388790,1,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
68,Central Toronto,43.696948,-79.411307,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
91,Downtown Toronto,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
